In [37]:
!apt install graphviz graphviz-dev
%pip install unified-planning[pyperplan,tamer,plot]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.42.2-6ubuntu0.1).
libgraphviz-dev is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [7]:
!wget https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/counters/domain.pddl -O /tmp/counters_domain.pddl
!wget https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/counters/problem.pddl -O /tmp/counters_problem.pddl

--2025-04-27 23:37:15--  https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/counters/domain.pddl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 843 [text/plain]
Saving to: ‘/tmp/counters_domain.pddl’

/tmp/counters_domai 100%[===================>]     843  --.-KB/s    in 0s      

2025-04-27 23:37:16 (44.9 MB/s) - ‘/tmp/counters_domain.pddl’ saved [843/843]

--2025-04-27 23:37:16--  https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/counters/problem.pddl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [38]:
!wget https://raw.githubusercontent.com/yuzhongchen/assignment/main/assignments/assignment-4/openrouter-domain.pddl -O openrouter-domain.pddl
!wget https://raw.githubusercontent.com/yuzhongchen/assignment/main/assignments/assignment-4/openrouter-problem.pddl -O openrouter-problem.pddl

--2025-04-28 00:10:52--  https://raw.githubusercontent.com/yuzhongchen/assignment/main/assignments/assignment-4/openrouter-domain.pddl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 485 [text/plain]
Saving to: ‘openrouter-domain.pddl’

openrouter-domain.p 100%[===================>]     485  --.-KB/s    in 0s      

2025-04-28 00:10:52 (27.1 MB/s) - ‘openrouter-domain.pddl’ saved [485/485]

--2025-04-28 00:10:52--  https://raw.githubusercontent.com/yuzhongchen/assignment/main/assignments/assignment-4/openrouter-problem.pddl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request 

In [41]:
import unified_planning
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *

reader = PDDLReader()
problem = reader.parse_problem('openrouter-domain.pddl', 'openrouter-problem.pddl')

# Now use the OneshotPlanner, which will have access to the registered engines
# Instead of passing 'environment' directly, use the names argument.
# The original code used names=['tamer', 'tamer', 'pyperplan'].
# This is changed to only use 'fast-downward' to see if the problem
# can be solved by a different engine.
with OneshotPlanner(name='pyperplan') as planner:
    result = planner.solve(problem)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Pyperplan returned: %s" % result.plan)
    else:
        print("No plan found.")

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 13 of `<ipython-input-41-1f3cbf897b80>`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

Pyperplan returned: SequentialPlan:
    move(l0, l1)
    move(l1, l2)
    move(l2, l3)
    move(l3, l4)
    move(l4, l5)
    move(l5, l6)
    move(l6, l7)
    move(l7, l8)
    move(l8, l9)


In [49]:
# robot_react_agent.py

from typing import List, Any, Dict
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.engines.results import POSITIVE_OUTCOMES

# ─── 0. INLINE TOOL & AGENT BASE CLASSES ─────────────────────────────────────

class Tool:
    """Base class for all tools."""
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def call(self, *args, **kwargs) -> Any:
        """Execute the tool. Override in subclasses."""
        raise NotImplementedError(f"{self.name}.call() not implemented")

class Agent:
    """Minimal ReAct‐style agent shell with tool‐calling support."""
    def __init__(self, tools: List[Tool]):
        self.tools: Dict[str, Tool] = {t.name: t for t in tools}

    def call_tool(self, tool_name: str, *args, **kwargs) -> Any:
        if tool_name not in self.tools:
            raise ValueError(f"Unknown tool: {tool_name}")
        return self.tools[tool_name].call(*args, **kwargs)

# ─── 1. TOOL: PDDL PLANNING (file‐based) ─────────────────────────────────────

class PDDLPlanningTool(Tool):
    def __init__(self):
        super().__init__(
            name="pddl_planner",
            description="Read PDDL files and compute a plan as a list of action strings."
        )

    def call(self, domain_path: str, problem_path: str) -> List[str]:
        # 1. Parse PDDL domain + problem from files
        reader = PDDLReader()
        problem = reader.parse_problem(domain_path, problem_path)

        # 2. Solve with OneshotPlanner (auto‐selects a compatible engine)
        with OneshotPlanner(problem_kind=problem.kind) as planner:
            result = planner.solve(problem)

        # 3. Return the sequence of ground actions, or empty if unsolvable
        if result.status in POSITIVE_OUTCOMES and result.plan is not None:
            # result.plan is a SequentialPlan; iterate its .actions
            return [str(a) for a in result.plan.actions]
        return []

# ─── 2. TOOL: ENVIRONMENT SIMULATOR ────────────────────────────────────────────

class EnvSimulatorTool(Tool):
    def __init__(self, initial: str):
        super().__init__(
            name="env_simulator",
            description="Simulate robot moves; track and update current location."
        )
        self.position = initial

    def call(self, action_step: str) -> str:
        # Remove parentheses and commas, then split
        cleaned = action_step.replace("(", " ") \
                             .replace(")", " ") \
                             .replace(",", " ")
        tokens = cleaned.split()
        if len(tokens) < 2:
            raise ValueError(f"Cannot parse move step: '{action_step}'")
        # Last two tokens are source and destination
        src, dst = tokens[-2], tokens[-1]
        if src != self.position:
            raise AssertionError(f"Precondition failed: at {self.position}, expected {src}")
        self.position = dst
        return self.position

# ─── 3. REACT AGENT IMPLEMENTATION ─────────────────────────────────────────────

class RobotAgent(Agent):
    def __init__(self, domain_file: str, problem_file: str, start_pos: str):
        super().__init__(tools=[
            PDDLPlanningTool(),
            EnvSimulatorTool(initial=start_pos)
        ])
        self.domain_file = domain_file
        self.problem_file = problem_file

    def run(self):
        # 1. Plan generation
        print("Thought: I need a plan to reach the goal from my current position.")
        print("Action: pddl_planner")
        plan_steps = self.call_tool("pddl_planner", self.domain_file, self.problem_file)
        print(f"Observation: Received plan → {plan_steps}")

        # 2. Execute plan steps
        for step in plan_steps:
            print(f"Thought: I will execute '{step}'.")
            print("Action: env_simulator")
            new_loc = self.call_tool("env_simulator", step)
            print(f"Observation: Robot moved to {new_loc}")

        print("Thought: I have reached the goal location. Task complete.")

# ─── 4. MAIN ENTRYPOINT ────────────────────────────────────────────────────────

if __name__ == "__main__":
    DOMAIN_FILE  = "openrouter-domain.pddl"
    PROBLEM_FILE = "openrouter-problem.pddl"
    agent = RobotAgent(DOMAIN_FILE, PROBLEM_FILE, start_pos="l0")
    agent.run()


Thought: I need a plan to reach the goal from my current position.
Action: pddl_planner
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 45 of `<ipython-input-49-b902d88e3076>`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

Observation: Received plan → ['move(l0, l1)', 'move(l1, l2)', 'move(l2, l3)', 'move(l3, l4)', 'move(l4, l5)', 'move(l5, l6)', 'move(l6, l7)', 'move(l7, l8)', 'move(l8, l9)']
Thought: I will execute 'move(l0, l1)'.
Action: env_simulator
Observation: Robot moved to l1
Thought: I will execute 'move(l1, l2)'.
Action: env_simulator
Observation: Robot moved to l2
Thought: I will execute 'move(l2, l3)'.
Action: env_sim